In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

Using TensorFlow backend.
/Users/lonetitan/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lonetitan/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lonetitan/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/lone

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [3]:
dirpath = os.getcwd()
train_dir = os.listdir(dirpath+"/train")

In [4]:
categories = []
for f_name in train_dir:
    category = f_name.split('.')[0]
    if category == "cat":
        categories.append(0)
    else:
        categories.append(1)

In [5]:
df = pd.DataFrame({"filename" : train_dir, "category"  :categories})

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Activation, BatchNormalization
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2,activation='softmax'))

In [11]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 128)      

In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [14]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=225)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [15]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "./train/",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./train/", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [16]:
train_df.head()

,filename,category
0,cat.11884.jpg,cat
1,dog.8372.jpg,dog
2,cat.5099.jpg,cat
3,cat.8115.jpg,cat
4,dog.3374.jpg,dog


In [17]:
train_df = train_df.reset_index(drop = True)
validate_df = validate_df.reset_index(drop = True)

In [18]:
train_df.head()

,filename,category
0,cat.11884.jpg,cat
1,dog.8372.jpg,dog
2,cat.5099.jpg,cat
3,cat.8115.jpg,cat
4,dog.3374.jpg,dog


In [19]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/10
1333/1333 [==============================] - 469s 352ms/step - loss: 0.6646 - acc: 0.6782 - val_loss: 0.6058 - val_acc: 0.7201
Epoch 2/10
1333/1333 [==============================] - 474s 355ms/step - loss: 0.4853 - acc: 0.7700 - val_loss: 0.3392 - val_acc: 0.7819
Epoch 3/10
1333/1333 [==============================] - 486s 365ms/step - loss: 0.4240 - acc: 0.8090 - val_loss: 0.9431 - val_acc: 0.6756
Epoch 4/10
1333/1333 [==============================] - 478s 358ms/step - loss: 0.3831 - acc: 0.8336 - val_loss: 0.3010 - val_acc: 0.8333
Epoch 5/10
1333/1333 [==============================] - 468s 351ms/step - loss: 0.3614 - acc: 0.8410 - val_loss: 0.2878 - val_acc: 0.8209
Epoch 6/10
1333/1333 [==============================] - 474s 356ms/step - loss: 0.3414 - acc: 0.8516 - val_loss: 0.4852 - val_acc: 0.8562
Epoch 7/10
1333/1333 [==============================] - 456s 342ms/step - loss: 0.3166 - acc: 0.8645 - val_loss: 0.3650 - val_acc: 0.8329
Epoch 8/10
1333/1333 [============

In [20]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [22]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
